In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
from ipywidgets import interact, Layout
import threading
import cvzone
import time
from sklearn.model_selection import train_test_split
import pickle

In [2]:
target_dict = {
    'Karta Polaka (FRONT)': 0,
    'Karta Polaka (BACK)': 1,
    'Driver License (FRONT)': 2,
    'Driver License (BACK)': 3,
              }

In [3]:
with open('model_0.pkl', 'rb') as file:  
    clf3 = pickle.load(file)

In [38]:
font = cv2.FONT_HERSHEY_SIMPLEX

width_img = 1280
height_img = 960

stopButton = widgets.ToggleButton(description='Stop', disabled=False)

val1 = widgets.IntRangeSlider(min=0, max=255, step=1, value=[30, 255], layout=Layout(width='80%'), description='Canny_threshold')
val2 = widgets.IntRangeSlider(min=0, max=255, step=1, value=[180, 255], layout=Layout(width='80%'))
val3 = widgets.RadioButtons(options=[True, False], description='L2Gradient', disabled=False)

b0 = [80,140,170,255,80,255]
b1_1 = [110,140,90,170,110,180]
b1_2 = [15,55,5,25,175,200]
b2 = [0,255,125,200,80,255]
pl = [0,255,175,255,95,235]
pl_2 = [87,115,39,77,128,207]

a = b1.copy()
hue = widgets.IntRangeSlider(min=0, max=255, step=1, value=[a[0], a[1]], layout=Layout(width='80%'), description='Hue')
sat = widgets.IntRangeSlider(min=0, max=255, step=1, value=[a[2], a[3]], layout=Layout(width='80%'), description='Saturation')
val = widgets.IntRangeSlider(min=0, max=255, step=1, value=[a[4], a[5]], layout=Layout(width='80%'), description='Value')

def preprocess(fr, v1):
    mask = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
    mask = cv2.GaussianBlur(mask, (5,5), 2)
    mask = cv2.Canny(mask, v1.value[0], v1.value[1], apertureSize=5)
	# mask = cv2.dilate(mask, np.ones((5,5)), iterations=2)
    # mask = cv2.erode(mask, np.ones((5,5)), iterations=1)
    return mask

def contours(fr_m, fr):
    biggest = np.array([])
    max_area = 0
    contours, hierarchy = cv2.findContours(fr_m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 100_000:
            epsilon = 0.1 * cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, epsilon, True)
            cv2.drawContours(fr, cnt, -1, (0,255,0), 5)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest

def reorder(my_points):
    my_points = my_points.reshape((4,2))
    my_points_new = np.zeros((4,1,2), np.int32)
    add = my_points.sum(1)

    my_points_new[0] = my_points[np.argmin(add)]
    my_points_new[3] = my_points[np.argmax(add)]

    diff = np.diff(my_points, axis=1)

    my_points_new[1] = my_points[np.argmin(diff)]
    my_points_new[2] = my_points[np.argmax(diff)]

    return my_points_new

def get_warp(img, approx):
    biggest = reorder(approx)

    pts1 = np.float32(biggest)
    pts2 = np.float32([[0, 0], [width_img, 0], [0, height_img], [width_img, height_img]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    img_output = cv2.warpPerspective(img, matrix, (width_img, height_img))
    img_cropped = cv2.resize(img_output, (width_img, height_img))
    return img_cropped

# CONTOURS CHECK
def text_detection(fr, img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    tmp = np.empty((1,4))
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = cv2.contourArea(contour)
        if area > 100:
            # cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            if tmp.shape[0] < 146: # 146 pr2[2]
                tmp = np.append(tmp, [[x, y, w, h]], axis=0)
                
    t = tmp[1:,:].astype(int)
    t = np.pad(t, [(0, 146 - t.shape[0]),(0, 0)], mode='constant').T.reshape(1, -1) # 146 pr2[2]
    ind = clf3.predict(t).argmax(1)[0]
    st = list(target_dict.keys())[list(target_dict.values()).index(ind)]
    cv2.putText(fr, st, (7, 70), font, 3, (0, 255, 0), 7, cv2.LINE_AA)
    return ind

def filters(fr, hue, sat, val):
    lower = np.array([hue.value[0], sat.value[0], val.value[0]])
    upper = np.array([hue.value[1], sat.value[1], val.value[1]])
    mask = cv2.cvtColor(fr, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(mask, lower, upper)
    # cv2.putText(fr, ind, (7, 70), font, 3, (0, 255, 0), 7, cv2.LINE_AA)
    return mask

def foto(img, hue, sat, val):
    # lower = np.array([hue.value[0], sat.value[0], val.value[0]])
    # upper = np.array([hue.value[1], sat.value[1], val.value[1]])
    # mask = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    # mask = cv2.inRange(mask, lower, upper)

    pl_0 = [0,255,0,255,250,255]
    pl_1 = [0,255,175,255,95,235]
    pl_2 = [87,115,39,77,128,207]
    pl_3 = [158,179,0,17,183,201]
    # l = [pl_1,pl_2,pl_3]
    l = [pl_1]

    # mask = cv2.threshold(img, 0, 0, cv2.THRESH_BINARY)[1]
    lower = np.array([0, 0, 255])
    upper = np.array([255, 255, 255])
    mask = cv2.inRange(img, lower, upper)

    for i in l:
        lower = np.array([i[0], i[2], i[4]])
        upper = np.array([i[1], i[3], i[5]])
        m = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        m = cv2.inRange(m, lower, upper)
        mask = cv2.addWeighted(mask,1,m,1,0)
        

    # lower1 = np.array([pl_1[0], pl_1[2], pl_1[4]])
    # upper1 = np.array([pl_1[1], pl_1[3], pl_1[5]])
    # mask1 = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    # mask1 = cv2.inRange(mask1, lower1, upper1)

    # lower2 = np.array([pl_2[0], pl_2[2], pl_2[4]])
    # upper2 = np.array([pl_2[1], pl_2[3], pl_2[5]])
    # mask2 = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    # mask2 = cv2.inRange(mask2, lower2, upper2)

    # mask = cv2.addWeighted(mask1,1,mask2,1,0)
    return mask
    

def view(button, val1, hue, sat, val):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)

    img = cv2.imread('id/11.jpg')

    while True:
        _, frame = cap.read()

        frame_masked = preprocess(frame, val1)
        frame_contours = contours(frame_masked, frame)

        f = foto(img, hue, sat, val)

        if frame_contours.size != 0:
            img_warped = get_warp(frame, frame_contours)
            frame_detect = text_detection(frame, img_warped)
            filt = filters(img_warped, hue, sat, val)
        else:
            img_warped = frame.copy()
            filt = frame.copy()
        img_stack = cvzone.stackImages([
            img,
            f,
            # frame,
            # filt,
            # img_warped,
        ], 2, 1)

        _, frame = cv2.imencode('.jpeg', img_stack) 
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)

display(stopButton, val1, hue, sat, val)
thread = threading.Thread(target=view, args=(stopButton, val1, hue, sat, val))
thread.start()

ToggleButton(value=False, description='Stop')

IntRangeSlider(value=(30, 255), description='Canny_threshold', layout=Layout(width='80%'), max=255)

IntRangeSlider(value=(7, 56), description='Hue', layout=Layout(width='80%'), max=255)

IntRangeSlider(value=(13, 69), description='Saturation', layout=Layout(width='80%'), max=255)

IntRangeSlider(value=(117, 255), description='Value', layout=Layout(width='80%'), max=255)

None

In [ ]:
# объеденить маленькие пересекающиеся зоны в одну большую
# ? делать диапазон inRange больше или меньше?